In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.optimize as sco

plt.style.use('seaborn-darkgrid')
sns.set(rc={'figure.figsize':(20,15)})
%matplotlib inline


In [2]:
%store -r ARIMA_pred_IVV
%store -r ARIMA_pred_EFA
%store -r ARIMA_pred_EEM
%store -r ARIMA_pred_VIG
%store -r ARIMA_pred_SHY
%store -r ARIMA_pred_LQD
%store -r ARIMA_pred_EMB
%store -r ARIMA_pred_MUB
%store -r ARIMA_pred_TIP
%store -r ARIMA_pred_VNQ
%store -r ARIMA_pred_XLE

In [3]:
ARIMA_pred = pd.concat([ARIMA_pred_IVV,ARIMA_pred_EFA,ARIMA_pred_EEM,ARIMA_pred_VIG,ARIMA_pred_SHY,
                        ARIMA_pred_LQD,ARIMA_pred_EMB,
                       ARIMA_pred_MUB,ARIMA_pred_TIP,ARIMA_pred_VNQ,ARIMA_pred_XLE], axis=1)

In [4]:
ARIMA_pred

,ARIMA_pred_IVV,ARIMA_pred_EFA,ARIMA_pred_EEM,ARIMA_pred_VIG,ARIMA_pred_SHY,ARIMA_pred_LQD,ARIMA_pred_EMB,ARIMA_pred_MUB,ARIMA_pred_TIP,ARIMA_pred_VNQ,ARIMA_pred_XLE
0,195.252081,58.344481,37.648886,74.525312,80.658847,102.489112,91.289412,98.392448,104.828442,66.044927,69.233829
1,194.089717,57.480314,36.672039,73.810961,80.536668,100.930345,89.557580,97.797108,103.388576,64.382676,68.637850
2,194.226878,57.915621,36.776607,74.045212,80.518342,100.656728,89.084448,97.900244,103.173265,64.412867,68.010754
3,195.404027,57.813802,36.621654,74.612409,80.636495,101.234318,89.728052,98.329140,104.135801,65.706896,67.068441
4,194.123748,57.559745,36.522372,73.726086,80.590632,100.205222,89.653675,98.162334,103.384701,63.638978,67.583771
5,192.917225,56.999897,36.625792,73.372487,80.680845,100.298610,89.609188,98.080804,103.760171,64.248911,66.197190
6,193.019602,57.245821,35.445193,73.718928,80.709042,100.348711,89.866706,98.476349,103.640289,65.523807,65.333624
7,197.074571,58.093228,35.647655,74.551935,80.691338,100.604783,89.766851,98.804437,103.720020,66.152880,64.432304
8,193.652046,56.955987,34.218156,73.231202,80.674340,100.979170,89.210366,99.160222,103.839466,65.861661,61.886521
9,195.452995,57.643452,34.109583,74.118374,80.718928,101.375746,89.615288,99.307117,104.296919,66.541582,61.759506


In [5]:
%store ARIMA_pred

Stored 'ARIMA_pred' (DataFrame)


In [6]:
ARIMA_pred.to_csv('ARIMA_pred.csv')

In [7]:
ARIMA_pred_pct = ARIMA_pred.pct_change(1).dropna()


In [8]:
ARIMA_pred_norm = ((ARIMA_pred_pct - ARIMA_pred_pct.mean()) / ARIMA_pred_pct.std()).dropna()
%store ARIMA_pred_norm

Stored 'ARIMA_pred_norm' (DataFrame)


In [25]:
returns_pred = ARIMA_pred.pct_change()
weighted_avg_ret = returns_pred.mean()
cov_matrix = returns_pred.cov()
runs = 50000
rf = 0.01

In [30]:
def portfolio_performance_annualized(weights, weighted_avg_ret, cov_matrix):
    returns_actual = np.sum(weighted_avg_ret*weights ) *52
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(52)
    return std, returns_actual

def random_portfolios(runs, weighted_avg_ret, cov_matrix, rf):
    results = np.zeros((3,runs))
    weights_record = []
    for i in range(runs):
        weights = np.random.random(11)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_performance_annualized(weights, weighted_avg_ret, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - rf) / portfolio_std_dev
    return results, weights_record

def simulated_pf_weights_only(weighted_avg_ret, cov_matrix, runs, rf):
    for i in range (9): 
        results, weights = random_portfolios(runs,weighted_avg_ret, cov_matrix, rf)

        max_sharpe_idx = np.argmax(results[2])
        sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
        max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=ARIMA_pred.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i,3)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T
        
        min_vol_idx = np.argmin(results[0])
        sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
        
        print('-'*80)
        print('Optimal Sharpe Ratio Portfolio Allocation')
        print(max_sharpe_allocation)

In [28]:
simulated_pf_weights_only(weighted_avg_ret, cov_matrix, runs, rf)

--------------------------------------------------------------------------------
Optimal Sharpe Ratio Portfolio Allocation
            ARIMA_pred_IVV  ARIMA_pred_EFA  ARIMA_pred_EEM  ARIMA_pred_VIG  \
allocation           0.075           0.004           0.005           0.053   

            ARIMA_pred_SHY  ARIMA_pred_LQD  ARIMA_pred_EMB  ARIMA_pred_MUB  \
allocation           0.126            0.25           0.193           0.218   

            ARIMA_pred_TIP  ARIMA_pred_VNQ  ARIMA_pred_XLE  
allocation           0.043           0.027           0.007  
--------------------------------------------------------------------------------
Optimal Sharpe Ratio Portfolio Allocation
            ARIMA_pred_IVV  ARIMA_pred_EFA  ARIMA_pred_EEM  ARIMA_pred_VIG  \
allocation           0.008            0.02           0.005           0.225   

            ARIMA_pred_SHY  ARIMA_pred_LQD  ARIMA_pred_EMB  ARIMA_pred_MUB  \
allocation            0.17           0.176           0.082           0.119   

    

In [31]:
def simulated_pf_weights_only2(weighted_avg_ret, cov_matrix, runs, rf):
    results, weights = random_portfolios(runs,weighted_avg_ret, cov_matrix, rf)

    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=ARIMA_pred.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i,3)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]

    print('-'*80)
    print('Optimal Sharpe Ratio Portfolio Allocation')
    print(max_sharpe_allocation)

In [32]:
simulated_pf_weights_only2(weighted_avg_ret, cov_matrix, runs, rf)

--------------------------------------------------------------------------------
Optimal Sharpe Ratio Portfolio Allocation
            ARIMA_pred_IVV  ARIMA_pred_EFA  ARIMA_pred_EEM  ARIMA_pred_VIG  \
allocation           0.031           0.011           0.004           0.168   

            ARIMA_pred_SHY  ARIMA_pred_LQD  ARIMA_pred_EMB  ARIMA_pred_MUB  \
allocation           0.229           0.148           0.214           0.175   

            ARIMA_pred_TIP  ARIMA_pred_VNQ  ARIMA_pred_XLE  
allocation           0.001           0.018           0.002  
